# Library

In [ ]:
!pip install gensim pyLDAvis nltk

In [88]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

from kiwipiepy import Kiwi
from kiwipiepy.utils import Stopwords
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

import gensim
from gensim import corpora

import pyLDAvis.gensim_models

# LDA

reference: [LDA](https://wikidocs.net/30708)

# Practice

## english news

In [13]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [3]:
data = pd.read_csv('./data/abcnews-date-text.csv')

In [9]:
# 각 기사 제목(headline_text)에 대해 단어를 토큰화하여 'tokens' 컬럼에 저장
data['tokens'] = data.headline_text.apply(lambda x: nltk.wordpunct_tokenize(x))

# NLTK의 영어 불용어(stop words) 리스트를 로드
stop_words = stopwords.words('english')

# 토큰 중 불용어를 제거하여 다시 'tokens' 컬럼에 저장
data.tokens = data.tokens.apply(lambda tokens: [token for token in tokens if token not in stop_words])

# 토큰들을 명사 형태로 표제어 추출(lemmatization)한 후, 다시 'tokens' 컬럼에 저장
data.tokens = data.tokens.apply(lambda tokens: [
    WordNetLemmatizer().lemmatize(token) for token in tokens if token not in stop_words
])

# 토큰들을 동사 형태로 표제어 추출(lemmatization)한 후, 다시 'tokens' 컬럼에 저장
data.tokens = data.tokens.apply(lambda tokens: [
    WordNetLemmatizer().lemmatize(token, pos='v') for token in tokens if token not in stop_words
])

# 길이가 3자 이상인 토큰만 남기고 다시 'tokens' 컬럼에 저장
data.tokens = data.tokens.apply(lambda tokens: [token for token in tokens if len(token) >= 3])


In [35]:
# 'data.tokens'에서 추출한 토큰들을 바탕으로 단어 사전을 생성
dictionary = corpora.Dictionary(data.tokens)

# 각 문서(기사)에 대해 단어의 등장 횟수를 기록하는 Bag of Words(BOW) 표현을 생성
# 각 문서는 (단어 ID, 단어 등장 횟수)의 리스트로 표현됨
corpus = [dictionary.doc2bow(text) for text in data.tokens]

# 네 번째 문서(세 번째 인덱스)에 대한 BOW 표현 출력
corpus[3]


[(15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1)]

In [42]:
# LDA(Latent Dirichlet Allocation) 모델을 학습
# - corpus: 문서의 BOW 표현
# - num_topics: 찾을 토픽의 개수 (여기서는 20개의 토픽을 찾음)
# - id2word: BOW에서 사용된 단어 ID와 실제 단어의 매핑을 제공하는 사전(dictionary)
lda_model = gensim.models.ldamodel.LdaModel(
    corpus,
    num_topics=20,
    id2word=dictionary,
)


In [45]:
# 각 토픽의 번호와 해당 토픽에 속하는 단어 5개를 출력
# - topic_num: 토픽 번호
# - tokens: 해당 토픽에서 가장 중요한 상위 5개의 단어와 그 중요도(확률) 출력
for topic_num, tokens in lda_model.print_topics(num_words=5):
    print(f'{topic_num}: {tokens}')

0: 0.064*"melbourne" + 0.044*"coast" + 0.039*"hospital" + 0.036*"health" + 0.034*"qld"
1: 0.044*"fight" + 0.043*"meet" + 0.041*"work" + 0.039*"tell" + 0.038*"return"
2: 0.081*"find" + 0.066*"queensland" + 0.054*"one" + 0.043*"state" + 0.036*"miss"
3: 0.053*"make" + 0.043*"market" + 0.043*"ban" + 0.042*"power" + 0.038*"share"
4: 0.070*"attack" + 0.039*"rise" + 0.038*"labor" + 0.034*"lose" + 0.033*"price"
5: 0.081*"year" + 0.040*"two" + 0.038*"jail" + 0.035*"life" + 0.033*"trial"
6: 0.062*"change" + 0.059*"back" + 0.040*"west" + 0.040*"record" + 0.030*"victorian"
7: 0.129*"australia" + 0.044*"canberra" + 0.039*"north" + 0.036*"brisbane" + 0.036*"fund"
8: 0.077*"election" + 0.072*"fire" + 0.065*"house" + 0.046*"donald" + 0.036*"student"
9: 0.161*"say" + 0.081*"nsw" + 0.051*"report" + 0.046*"show" + 0.034*"country"
10: 0.076*"police" + 0.074*"man" + 0.057*"woman" + 0.056*"charge" + 0.051*"court"
11: 0.092*"sydney" + 0.040*"tasmanian" + 0.038*"indigenous" + 0.028*"city" + 0.025*"three"
12: 

In [46]:
# Jupyter Notebook에서 pyLDAvis 시각화 활성화
pyLDAvis.enable_notebook()

# pyLDAvis로 LDA 모델 시각화를 준비
# - lda_model: 학습된 LDA 모델
# - corpus: 문서들의 BOW 표현
# - dictionary: 단어 사전 (단어 ID와 실제 단어의 매핑)
visualization = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary)

# LDA 토픽 모델의 시각화 출력
pyLDAvis.display(visualization)


In [61]:
# 각 문서에 대해 LDA 모델이 할당한 토픽 번호를 추출하여 데이터프레임에 저장
for i, corpus_ in enumerate(corpus):
    # 각 문서에 대해 토픽 분포를 계산
    result = lda_model[corpus_]
    
    # 각 문서에서 가장 확률이 높은(가장 중요한) 토픽 번호를 찾음
    # result는 (토픽 번호, 해당 토픽의 확률) 형태의 리스트이므로, 확률 기준으로 내림차순 정렬 후 첫 번째 항목 선택
    topic_num, _ = sorted(result, key=lambda x: x[1], reverse=True)[0]
    
    # 해당 문서의 행에 가장 높은 확률의 토픽 번호를 'topic_num' 컬럼에 저장
    data.loc[i, 'topic_num'] = topic_num


In [65]:
data

,publish_date,headline_text,tokens,topic_num
0,20030219,aba decides against community broadcasting lic...,"[aba, decide, community, broadcast, licence]",15.0
1,20030219,act fire witnesses must be aware of defamation,"[act, fire, witness, must, aware, defamation]",19.0
2,20030219,a g calls for infrastructure protection summit,"[call, infrastructure, protection, summit]",6.0
3,20030219,air nz staff in aust strike for pay rise,"[air, staff, aust, strike, pay, rise]",4.0
4,20030219,air nz strike to affect australian travellers,"[air, strike, affect, australian, traveller]",3.0
...,...,...,...,...
1082163,20170630,when is it ok to compliment a womans smile a g...,"[compliment, woman, smile, guide]",10.0
1082164,20170630,white house defends trumps tweet,"[white, house, defend, trump, tweet]",8.0
1082165,20170630,winter closes in on tasmania as snow ice falls,"[winter, close, tasmania, snow, ice, fall]",15.0
1082166,20170630,womens world cup australia wins despite atapat...,"[womens, world, cup, australia, win, despite, ...",16.0


## 기자회견 

In [75]:
comments = pd.read_pickle('./data/comments_minheejin.pickle')

kiwi = Kiwi()
# 단어 사전에 단어 추가
kiwi.add_user_word('어도어', 'NNP')
kiwi.add_user_word('빌리프랩', 'NNP')
kiwi.add_user_word('빌리프렙', 'NNP')
kiwi.add_user_word('아일리스', 'NNP')
kiwi.add_user_word('르세라핌', 'NNP')
kiwi.add_user_word('피프티피프티', 'NNP')
kiwi.add_user_word('뉴진스', 'NNP')
kiwi.add_user_word('아일릿', 'NNP')
kiwi.add_user_word('하이브', 'NNP')
kiwi.add_user_word('방시혁', 'NNP')
kiwi.add_user_word('힛뱅맨', 'NNP')
kiwi.add_user_word('힛맨뱅', 'NNP')
kiwi.add_user_word('민희진', 'NNP')
kiwi.add_user_word('미니진', 'NNP')
kiwi.add_user_word('희진', 'NNP')
kiwi.add_user_word('진짜사나이이', 'NNP')
kiwi.add_user_word('레퍼런스', 'NNG')
kiwi.add_user_word('언플', 'NNG')
kiwi.add_user_word('대퓨', 'NNG')
kiwi.add_user_word('대퓨님', 'NNG')
kiwi.add_user_word('개저씨', 'NNG')
kiwi.add_user_word('댓글부대', 'NNG')

# 불용어 사전에 불용어 추가
stopwords = Stopwords()
stopwords.add(('웩퉥', 'NNG'))
stopwords.add(('결국', 'NNG'))

def extract_tokens(string: str, tokenizer: Kiwi, stopwords: Stopwords, tags={'NNP', 'NNG'}):
    # 주어진 문자열(string)을 입력으로 받아, 지정된 품사 태그와 길이 조건을 만족하는 토큰들을 추출하는 함수

    # Kiwi 객체를 사용하여 문자열을 토크나이즈(tokenize)하고, 불용어(stopwords)를 적용
    tokens = tokenizer.tokenize(string, stopwords=stopwords)
    
    # 토크나이즈된 토큰 중에서, 지정된 태그 집합(tags)에 포함되며 길이가 2 이상인 토큰들의 형태소를 추출하여 리스트에 저장
    target_tokens = [token.form for token in tokens if token.tag in tags and len(token.form) >= 2]

    # 조건을 만족하는 토큰들의 리스트를 반환
    return target_tokens

In [76]:
# comments 데이터프레임의 'textOriginal' 열에 있는 텍스트에 대해
# extract_tokens 함수를 적용하여 추출된 토큰들을 'tokens' 열에 저장
comments['tokens'] = comments.textOriginal.apply(lambda x: extract_tokens(x, kiwi, stopwords))

# 'tokens' 열에 있는 토큰 리스트를 문자열로 변환(join)하여, 
# 각각의 리스트를 공백으로 구분된 문자열로 합치고, 이를 리스트로 변환하여 'tokens' 변수에 저장
tokens = comments.tokens.str.join(' ').tolist()  # 혹은 comments.tokens.apply(lambda x: ' '.join(x))

In [77]:
# 'data.tokens'에서 추출한 토큰들을 바탕으로 단어 사전을 생성
dictionary = corpora.Dictionary(comments.tokens)

# 각 문서(기사)에 대해 단어의 등장 횟수를 기록하는 Bag of Words(BOW) 표현을 생성
# 각 문서는 (단어 ID, 단어 등장 횟수)의 리스트로 표현됨
corpus = [dictionary.doc2bow(text) for text in comments.tokens]


In [80]:
lda_model = gensim.models.ldamodel.LdaModel(
    corpus,
    num_topics=30,
    id2word=dictionary,
    passes=10,
)

In [81]:
# Jupyter Notebook에서 pyLDAvis 시각화 활성화
pyLDAvis.enable_notebook()

# pyLDAvis로 LDA 모델 시각화를 준비
# - lda_model: 학습된 LDA 모델
# - corpus: 문서들의 BOW 표현
# - dictionary: 단어 사전 (단어 ID와 실제 단어의 매핑)
visualization = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary)

# LDA 토픽 모델의 시각화 출력
pyLDAvis.display(visualization)


## 사업의 개요

In [ ]:
data['business_info'] = data['사업의 개요'].apply(lambda x: BeautifulSoup(x, 'lxml').text)
data.business_info = data.business_info.str.replace('\s+', ' ', regex=True)
data.business_info = data.business_info.str.replace('\d?\.? ? 사업의 개요 ?', '', regex=True)

In [91]:
def extract_tokens(string: str, tokenizer: Kiwi, stopwords: Stopwords, tags={'NNP', 'NNG'}):
    # 주어진 문자열(string)을 입력으로 받아, 지정된 품사 태그와 길이 조건을 만족하는 토큰들을 추출하는 함수

    # Kiwi 객체를 사용하여 문자열을 토크나이즈(tokenize)하고, 불용어(stopwords)를 적용
    tokens = tokenizer.tokenize(string, stopwords=stopwords)
    
    # 토크나이즈된 토큰 중에서, 지정된 태그 집합(tags)에 포함되며 길이가 2 이상인 토큰들의 형태소를 추출하여 리스트에 저장
    target_tokens = [token.form for token in tokens if token.tag in tags and len(token.form) >= 2]

    # 조건을 만족하는 토큰들의 리스트를 반환
    return target_tokens

kiwi = Kiwi()
kiwi.add_user_word('바이오시밀러', 'NNG')

stopwords = Stopwords()
stopwords.add(('당사', 'NNG'))
stopwords.add(('산업', 'NNG'))
stopwords.add(('특성', 'NNG'))
stopwords.add(('웅진씽크빅', 'NNP'))

data['tokens'] = data.business_info.apply(lambda x: extract_tokens(x, kiwi, stopwords))
data['tokens_joined'] = data.tokens.str.join(' ')

In [92]:
# 'data.tokens'에서 추출한 토큰들을 바탕으로 단어 사전을 생성
dictionary = corpora.Dictionary(data.tokens)

# 각 문서(기사)에 대해 단어의 등장 횟수를 기록하는 Bag of Words(BOW) 표현을 생성
# 각 문서는 (단어 ID, 단어 등장 횟수)의 리스트로 표현됨
corpus = [dictionary.doc2bow(text) for text in data.tokens]

# LDA(Latent Dirichlet Allocation) 모델을 학습
# - corpus: 문서의 BOW 표현
# - num_topics: 찾을 토픽의 개수 (여기서는 20개의 토픽을 찾음)
# - id2word: BOW에서 사용된 단어 ID와 실제 단어의 매핑을 제공하는 사전(dictionary)
lda_model = gensim.models.ldamodel.LdaModel(
    corpus,
    num_topics=30,
    id2word=dictionary,
)

# Jupyter Notebook에서 pyLDAvis 시각화 활성화
pyLDAvis.enable_notebook()

# pyLDAvis로 LDA 모델 시각화를 준비
# - lda_model: 학습된 LDA 모델
# - corpus: 문서들의 BOW 표현
# - dictionary: 단어 사전 (단어 ID와 실제 단어의 매핑)
visualization = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary)

# LDA 토픽 모델의 시각화 출력
pyLDAvis.display(visualization)
